<a href="https://colab.research.google.com/github/SidneiFeitosa/Simulador-Agentes/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%pip -q install google-genai

In [28]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [29]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [30]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias **12 e 16 de maio de 2025**. As inscrições estão abertas até o dia 11 de maio.


In [31]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu de 12 a 16 de maio de 2025. As inscrições estavam abertas até o dia 11 de maio de 2025.


In [32]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima imersão IA Alura Google Gemini', 'Alura imersão IA Google Gemini datas']
Páginas utilizadas na resposta: thallesbenicio.com.br, eucapacito.com.br, convergenciadigital.com.br, tecmundo.com.br, youtube.com, alura.com.br



In [33]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [36]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [38]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [41]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [43]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [45]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [56]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [57]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [58]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ia
Maravilha! Vamos então criar o post sobre novidades em ia

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos de IA mais relevantes e atuais, focarei em notícias e reações entusiasmadas das últimas semanas.
> 
> 
> Com base nas notícias e tendências atuais, aqui estão 5 lançamentos e desenvolvimentos de IA que parecem ser os mais relevantes e entusiasmantes até maio de 2025:
> 
> 1.  **OpenAI GPT-4.1 e GPT-4.1 Mini:** A OpenAI lançou os modelos GPT-4.1 e GPT-4.1 Mini para usuários do ChatGPT. O GPT-4.1 promete um desempenho mais rápido e melhorias em tarefas de programação, enquanto o GPT-4.1 Mini está acessível a todos os usuários da plataforma.
> 2.  **IA Generativa Multimodal:** A Meta AI lançou a família de modelos Llama 4, apresentando a IA nativamente multimodal. Os modelos Llama 4 Scout e Llama 4 Maverick são projetados com fusão inicial, pré-treinados com dados textuais e visuais, resultando em uma compreensão mais natural e intuitiva através de diferentes modalidades.
> 3.  **Google Gemini:** O Gemini, modelo de IA do Google, está expandindo sua presença para plataformas como Wear OS, Android Auto e Google TV. Além disso, a Google está aprofundando a integração da IA na sua ferramenta de pesquisa e desenvolvendo uma plataforma alternativa ao Pinterest com capacidades de inteligência artificial.
> 4.  **Scientist AI:** Um novo modelo de IA que promete guardrails para prevenir comportamentos autônomos indesejados. A abordagem Scientist AI se baseia em um modelo de compreensão causal do mundo, priorizando a honestidade e apresentando uma alternativa segura à trajetória atual de IA baseada na autonomia.
> 5.  **Automação Inteligente de Processos:** A automação inteligente, impulsionada por IA, está revolucionando processos diversos ao simplificar, automatizar e otimizar operações de maneira mais estratégica. A combinação de aprendizado de máquina, análise avançada e algoritmos permite eliminar tarefas manuais repetitivas, reduzir custos operacionais e elevar a eficiência das operações.
> 
> Esses lançamentos representam avanços significativos em diversas áreas da IA, desde modelos de linguagem e geração multimodal até aplicações em segurança, automação e saúde. Eles demonstram o rápido crescimento e a crescente importância da IA em vários setores da sociedade.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos e desenvolvimentos de IA que você listou, vou criar um plano de conteúdo detalhado para cada um deles e, em seguida, selecionar o tema mais relevante para um post.
> 
> **1. OpenAI GPT-4.1 e GPT-4.1 Mini:**
> 
> *   **Pontos Relevantes:**
>     *   Lançamento dos modelos GPT-4.1 e GPT-4.1 Mini.
>     *   GPT-4.1: Desempenho mais rápido e melhorias em programação.
>     *   GPT-4.1 Mini: Acessível a todos os usuários do ChatGPT.
> *   **Plano de Conteúdo:**
>     *   Explicar o que são os modelos GPT-4.1 e GPT-4.1 Mini.
>     *   Detalhar as melhorias de desempenho do GPT-4.1, com foco em programação.
>     *   Destacar a acessibilidade do GPT-4.1 Mini e como isso democratiza o acesso à IA.
>     *   Exemplos de casos de uso para cada modelo.
> 
> **2. IA Generativa Multimodal (Meta AI - Llama 4):**
> 
> *   **Pontos Relevantes:**
>     *   Lançamento da família de modelos Llama 4.
>     *   IA nativamente multimodal.
>     *   Modelos Llama 4 Scout e Llama 4 Maverick.
>     *   Fusão inicial de dados textuais e visuais.
>     *   Compreensão mais natural e intuitiva.
> *   **Plano de Conteúdo:**
>     *   Introduzir o conceito de IA multimodal.
>     *   Apresentar a família Llama 4 e seus modelos.
>     *   Explicar a importância da fusão inicial de dados textuais e visuais.
>     *   Exemplificar como a IA multimodal melhora a compreensão em diferentes contextos.
>     *   Casos de uso práticos da IA multimodal.
> 
> **3. Google Gemini:**
> 
> *   **Pontos Relevantes:**
>     *   Expansão para Wear OS, Android Auto e Google TV.
>     *   Integração aprofundada na ferramenta de pesquisa do Google.
>     *   Desenvolvimento de plataforma alternativa ao Pinterest com IA.
> *   **Plano de Conteúdo:**
>     *   Apresentar o Google Gemini e seu potencial.
>     *   Detalhar a expansão para diferentes plataformas (Wear OS, Android Auto, Google TV).
>     *   Explicar como a IA está sendo integrada na busca do Google.
>     *   Apresentar a plataforma alternativa ao Pinterest e suas funcionalidades de IA.
>     *   Discutir o impacto da IA do Google no futuro da busca e entretenimento.
> 
> **4. Scientist AI:**
> 
> *   **Pontos Relevantes:**
>     *   Novo modelo de IA com "guardrails" para prevenir comportamentos indesejados.
>     *   Baseado em um modelo de compreensão causal do mundo.
>     *   Prioriza a honestidade.
>     *   Alternativa segura à IA baseada na autonomia.
> *   **Plano de Conteúdo:**
>     *   Introduzir o conceito de "guardrails" em IA e sua importância.
>     *   Apresentar o Scientist AI como uma solução para prevenir comportamentos autônomos indesejados.
>     *   Explicar o modelo de compreensão causal do mundo.
>     *   Destacar a priorização da honestidade e segurança.
>     *   Debater a necessidade de alternativas seguras à IA autônoma.
> 
> **5. Automação Inteligente de Processos:**
> 
> *   **Pontos Relevantes:**
>     *   Revolução de processos diversos.
>     *   Simplificação, automação e otimização de operações.
>     *   Combinação de aprendizado de máquina, análise avançada e algoritmos.
>     *   Eliminação de tarefas manuais repetitivas.
>     *   Redução de custos operacionais e aumento da eficiência.
> *   **Plano de Conteúdo:**
>     *   Definir o que é Automação Inteligente de Processos (IPA).
>     *   Explicar como a IPA está transformando diferentes setores.
>     *   Detalhar as tecnologias envolvidas (aprendizado de máquina, análise avançada, algoritmos).
>     *   Apresentar exemplos de sucesso de IPA em empresas.
>     *   Discutir os benefícios da IPA (redução de custos, aumento da eficiência, etc.).
> 
> **Escolha do Tema Mais Relevante:**
> 
> Após analisar os temas, considero que o **Scientist AI** é o mais relevante para um post no momento. A questão da segurança e ética na IA é crucial e está gerando muitos debates. Um modelo que prioriza a honestidade e busca prevenir comportamentos indesejados é extremamente interessante e pode atrair muita atenção.
> 
> **Plano Detalhado para o Post sobre Scientist AI:**
> 
> *   **Título:** "Scientist AI: A IA Honesta que Pode Mudar o Futuro da Autonomia"
> *   **Introdução:**
>     *   Contextualizar a crescente preocupação com a segurança e ética na IA.
>     *   Apresentar o Scientist AI como uma solução inovadora.
> *   **O que é Scientist AI:**
>     *   Explicar que é um novo modelo de IA com "guardrails" para prevenir comportamentos autônomos indesejados.
>     *   Detalhar que ele é baseado em um modelo de compreensão causal do mundo.
> *   **Por que a Honestidade é Importante:**
>     *   Discutir os riscos de uma IA autônoma sem valores éticos.
>     *   Explicar como o Scientist AI prioriza a honestidade em suas decisões.
> *   **Como o Scientist AI Funciona:**
>     *   Detalhar o modelo de compreensão causal do mundo.
>     *   Explicar como os "guardrails" previnem comportamentos indesejados.
> *   **Aplicações e Impacto:**
>     *   Discutir possíveis aplicações do Scientist AI em diferentes setores.
>     *   Analisar o impacto potencial na segurança e ética da IA.
> *   **Conclusão:**
>     *   Reafirmar a importância de modelos de IA seguros e éticos.
>     *   Incentivar a discussão sobre o futuro da IA e a necessidade de "guardrails".
> *   **Call to Action:**
>     *   Convidar os leitores a compartilhar suas opiniões sobre o Scientist AI e a segurança da IA.
> 
> Agora que temos um plano detalhado, podemos criar um post envolvente e informativo sobre o Scientist AI!
> 


--------------------------------------------------------------


...


--- 📝 Resultado do Agente 3 (Redator) ---



--------------------------------------------------------------


Exception in thread Thread-38 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
